In [10]:

####################################################################################################
####################################################################################################

import sys
import os

#import glob
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint


#from IPython.display import display

#
sys.path.append(os.path.abspath("../"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils

sys.path.append(os.path.abspath("../pax"))
from pax import core

pp = pprint.PrettyPrinter(depth=6)



In [11]:

####################################################################################################
####################################################################################################

zipfilename = '../pax_run/pax_output/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'


####################################################################################################
####################################################################################################

nEventsPerFile = 5
lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)



In [17]:

####################################################################################################
####################################################################################################

df_zip_event  = pd.DataFrame()
df_zip_intr   = pd.DataFrame()
df_zip_merged = pd.DataFrame()

    
####################################################################################################
####################################################################################################

for iPklFile in range(0, len(lst_pkl_files)):

    ################################################################################################
    ################################################################################################
    
    pklfilename  = lst_pkl_files[iPklFile]
    event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

    
    ################################################################################################
    ################################################################################################
    
    interactions = event.interactions
    
    if (len(interactions) > 0):
        
        intr = interactions[0]
        
        #intr.
    
    
    ################################################################################################
    ################################################################################################
    
    df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
    df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
    df_pkl_merged = pd.merge(df_pkl_event, df_pkl_intr)
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_event  = df_zip_event.append(df_pkl_event)
    df_zip_intr   = df_zip_intr.append(df_pkl_intr)
    df_zip_merged = df_zip_merged.append(df_pkl_merged.copy())
    
    continue
    
        

####################################################################################################
####################################################################################################

df_zip_merged.reset_index(inplace=True, drop=True)


####################################################################################################
####################################################################################################

#pp.pprint(list(df_zip_merged.columns.values))

print()
display(df_zip_merged)
print()

   

,event_number,event_n_pulses,event_pulses_count,event_s1_count,event_s2_count,intr_count,intr_corrected_s1_area,intr_corrected_s2_area,intr_log_cs2_cs1,intr_drift_time,...,intr_xy_posrec_algorithm,intr_xy_posrec_goodness_of_fit,intr_xy_posrec_ndf,intr_y,intr_z,intr_z_correction,intr_x_nn,intr_y_nn,reco_s1_photons,reco_s2_electrons
0,0,190,190,0,3,0,32.996575,1998.884861,NaN,402780.0,...,PosRecTopPatternFit,204.862053,112.0,-5.610902,-57.75552,0.0,0,0,0.0,93.844360
1,1,400,400,1,6,1,19.432068,2102.227029,NaN,400890.0,...,PosRecTopPatternFit,164.851306,112.0,-16.583333,-57.48336,0.0,0,0,0.0,98.696105
2,2,223,223,0,5,0,19.432068,2102.227029,NaN,400890.0,...,PosRecTopPatternFit,164.851306,112.0,-16.583333,-57.48336,0.0,0,0,0.0,98.696105
3,3,516,516,1,11,1,32.996575,1998.884861,NaN,402780.0,...,PosRecTopPatternFit,204.862053,112.0,-5.610902,-57.75552,0.0,0,0,0.0,93.844360
4,4,245,245,0,5,0,32.996575,1998.884861,NaN,402780.0,...,PosRecTopPatternFit,204.862053,112.0,-5.610902,-57.75552,0.0,0,0,0.0,93.844360
